# Handy
## Select ROI

ROI (region of interest) is a term of computer vision that defines the portion of an image that will be processed. In Handy, ROI defines the area the detected person has to stand in to be considered as target.

In [6]:
import sys
sys.path.append('..')
from config import CONFIG

In [2]:
import json
import logging
from os import path
import cv2

rectangle = []
drawing = False

HANDY_WINDOW = "Handy"

def draw_rectangle(event, x, y, flags, param):
    global rectangle
    global drawing
    if event == cv2.EVENT_LBUTTONDOWN:
        # Left mouse button down - store the starting point
        drawing = True
        rectangle = [x, y, x, y]
    elif event == cv2.EVENT_MOUSEMOVE:
        # Move has moved - update the rectangle
        if drawing:
            rectangle[2], rectangle[3] = x, y
    elif event == cv2.EVENT_LBUTTONUP:
        # Left mouse button up - end drawing
        drawing = False


# Init the video feed
cap = cv2.VideoCapture(CONFIG.stream_url, cv2.CAP_FFMPEG)

if not cap.isOpened():
    logging.error("VideoCapture not opened")
    exit(-1)

cv2.namedWindow(HANDY_WINDOW)
cv2.setMouseCallback(HANDY_WINDOW, draw_rectangle)

while True:
    ret, frame = cap.read()
    if not ret or frame is None:
        print("Corrupted frame, retrying...")
        continue

    # Resize frame
    frame = cv2.resize(frame, (CONFIG.resize_width, CONFIG.resize_height))

    cv2.putText(frame, "When you're ready, press 'q'", (0, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 3)
    # If there's a full rectangle
    if len(rectangle) == 4:
        cv2.rectangle(frame, (rectangle[0], rectangle[1]), (rectangle[2], rectangle[3]), (0, 255, 0), 2)
    
    cv2.imshow(HANDY_WINDOW, frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

if rectangle:
    print(f"Selected: {rectangle}")

    with open(path.join("..", "ROI.json"), "w") as f:
        ROI = {
            "x1": rectangle[0],
            "y1": rectangle[1],
            "x2": rectangle[2],
            "y2": rectangle[3],
        }
        json.dump(ROI, f)
else:
    print("Didn't select the ROI!")

cap.release()
cv2.destroyAllWindows()


: 